In [28]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
openai = OpenAI()

In [29]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Math expression like '12 * 5 + 3'"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_population",
            "description": "Get population information for a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "Population Count"
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for general information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query"
                    }
                },
                "required": ["query"]
            }
        }
    }
]


In [30]:
def get_population(city: str) -> int:
    """
    Mock population lookup
    """
    population_db = {
        "mumbai": 12442373,
        "pune": 3124458,
        "delhi": 16787941,
        "bangalore": 8443675
    }

    print("inside get_population function", city, city.lower() not in population_db)
    if city.lower() not in population_db:
        raise ValueError("City not found")

    return population_db.get(city.lower(), 0)

def get_weather(city: str) -> str:
    weather_db = {
        "mumbai": "Sunny, 32°C",
        "pune": "Cloudy, 28°C",
        "bangalore": "Rainy, 24°C",
        "delhi": "Hazy, 35°C"
    }
    return weather_db.get(city.lower(), "Weather data not available")

def calculator(expression: str) -> str:
    try:
        result = eval(expression, {"__builtins__": {}}, math.__dict__)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"


In [31]:
import time

def retry_tool(fn, *, retries=2, delay=0.5, **kwargs):
    last_error = None

    for attempt in range(retries + 1):
        try:
            return fn(**kwargs)
        except Exception as e:
            last_error = e
            if attempt < retries:
                time.sleep(delay)

    raise last_error


def execute_tool_with_fallback(tool_name, args):
    try:
        if tool_name == "get_population":
            return retry_tool(
                get_population,
                city=args["city"]
            )

        if tool_name == "calculator":
            return retry_tool(
                calculator,
                expression=args["expression"]
            )

    except Exception as e:
        # 🔻 graceful fallback result
        return {
            "error": True,
            "message": f"{tool_name} failed: {str(e)}"
        }


In [32]:
import json

def calling_agent_with_tools(user_input: str):

    messages = [
        {"role": "system", "content": '''You are a helpful assistant that uses tools when needed. 
        If a tool fails, don't assume any response, simply show that you are not able to compute the required answer
        Do Not Estimate anything if the tool fails
        '''},
        {"role": "user", "content": user_input}
    ]

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools
    )
    message = response.choices[0].message

    print("Model message:", message)

    # ✅ Append assistant message ONCE
    messages.append(message)

    if message.tool_calls:
        for tool_call in message.tool_calls:
            fn_name = tool_call.function.name
            fn_args = json.loads(tool_call.function.arguments)

            print(f"Calling tool: {fn_name} with args: {fn_args}")

            result = execute_tool_with_fallback(fn_name, fn_args)

            # 🔔 User-visible degradation signal
            if isinstance(result, dict) and result.get("error"):
                output = (
                    f"⚠️ I couldn't complete `{fn_name}` reliably. "
                    f"Reason: {result['message']}"
                )
            else:
                output = str(result)

            print("result", result)
            print("output", output)
            # ✅ Respond to EACH tool_call_id
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": fn_name,
                "content": str(output)
            })

        # ✅ Call model ONLY AFTER all tools are handled
        final_response = openai.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages
        )

        return final_response.choices[0].message.content

    return message.content

# print(calling_agent_with_tools("What is the population of Mumbai? Also, what is 25 multiplied by 4 plus 10?"))
print(calling_agent_with_tools("What is the population of new York city??"))

Model message: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_iu7rgU5Qwq6kGChgmJYputEL', function=Function(arguments='{"city":"New York City"}', name='get_population'), type='function')])
Calling tool: get_population with args: {'city': 'New York City'}
inside get_population function New York City True
inside get_population function New York City True
inside get_population function New York City True
result {'error': True, 'message': 'get_population failed: City not found'}
output ⚠️ I couldn't complete `get_population` reliably. Reason: get_population failed: City not found
I'm sorry, I couldn't retrieve the current population of New York City at the moment. However, as of recent estimates, New York City has a population of approximately 8.5 million people. For the most accurate and updated information, you might want to check official sources like the U.S. C